In [1]:
import plotly.offline as plt
from plotly import tools
import plotly.graph_objs as go
from plotly.graph_objs import Layout
from plotly.graph_objs.graph_objs import Scatter, Data
from ipywidgets import *
import ipywidgets as widgets
import math

plt.init_notebook_mode(connected=True)

In [4]:
class MapToD(object):
    """Class for chowing 2D map for N and H chemical shifts per steps"""
    def __init__(self, titration):
        """Defines ettributs
        argument 'titration' is a dicitionnary of 2 dimensions list for each key (should be a titration object from shiftome after developement)"""
        self.titration=titration
        self.selector=widgets.SelectMultiple(options=[key for key in self.titration],
                                             rows=len(self.titration),
                                             description='Residue ',
                                             disabled=False) # multiple selector widget for selecting one or more residues
    def scatterPlot(self, residues):
        """Method that traces scatter plot of N and H chemical shift per step for one or more residues."""
        if residues == (): #no residues selected
            pass
        else:
            data=[]
            rows=math.ceil(math.sqrt(len(residues))) # defines appropriate number of rows 
            cols=round(math.sqrt(len(residues))) # defines appropraite number of columns
            fig = tools.make_subplots(rows=rows, cols=cols, print_grid=False)
            colNumber=0
            rowNumber=1
            for res in residues:
                ##by-columns-then-by-lines method for plot positionning
                if colNumber < cols:
                    colNumber+=1
                elif rowNumber < rows:
                    colNumber =1
                    rowNumber +=1
                ##
                trace=go.Scatter(x=self.titration[res][0], 
                                 y=self.titration[res][1],
                                 mode='markers', 
                                 name='Residue '+str(res), 
                                 text=["Step "+str(i+1) for i in range(len(self.titration[res][0]))],
                                 marker={'color':[index+1 for index in range(len(self.titration[res][0]))], # sets a color scale regarding step number
                                         'colorbar':{'title':'Steps', 'titlefont':{'size':16}},
                                         'colorscale':'Viridis'}) 
                data.append(trace)
                fig.append_trace(trace, rowNumber,colNumber)
            fig['layout'].update(title='H and N chemical shifts of residue '+", ".join(map(str, residues)),
                                 showlegend=False,
                                        annotations=[{
                                                'x':1, #x posiition of annotation
                                                'y':5,
                                                'xref':'x', #plot number
                                                'yref':'y',
                                                'text':'max=5',
                                                'showarrow':False,
                                                'font':{
                                                    'family':'Courier New, monospace',
                                                    'size':11,
                                                    'color':'#ffffff'
                                                },

                                                'bordercolor':'rgb(255,255,255)',
                                                'borderwidth':2,
                                                'borderpad':4,
                                                'bgcolor':'rgb(0,0,0)',
                                                'opacity':0.7
                                        }
                                        ]*len(residues)+[{
                                                'xref': 'paper',
                                                'yref': 'paper',
                                                'x': 0, #x position of x ax legend
                                                'xanchor': 'right',
                                                'y': 1,
                                                'yanchor': 'bottom',
                                                'text': 'N Chemical shift',
                                                'font':{'size':16},
                                                'showarrow': False
                                              }, {
                                                'xref': 'paper',
                                                'yref': 'paper',
                                                'x': 1,
                                                'xanchor': 'left',
                                                'y': 0,
                                                'yanchor': 'top',
                                                'text': 'H chemical shift',
                                                'font':{'size':16},
                                                'showarrow': False
                                              }],
                                        width=1000, #width of plotly interface
                                        height=1000,
                                        margin=go.Margin(
                                            l=200,
                                            r=200,
                                            b=100,
                                            t=100,
                                            pad=4
        )
                    )
            for index,res in enumerate(residues):
                fig['layout']['annotations'][index]['xref']="x"+str(index+1) #annotation affectation
                fig['layout']['annotations'][index]['yref']="y"+str(index+1)
                fig['layout']['annotations'][index]['text']="Residue " +str(res)
                fig['layout']['annotations'][index]['x']=(max(self.titration[res][0])+min(self.titration[res][0]))/2 #annotation: middle plot affectation
                fig['layout']['annotations'][index]['y']=1.2*max(self.titration[res][1])#annotation: top plot affectation
                fig['layout']['xaxis'+str(index+1)].update(showline=True,gridwidth=5) #show x ax
                fig['layout']['yaxis'+str(index+1)].update(showline=True,gridwidth=5)
                
            plt.iplot(fig)
                 
titrationObj={1:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 2:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],3:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 4:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],5:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 6:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],8:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 9:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]]}          
dd=MapToD(titrationObj)
widgets.interactive(dd.scatterPlot, residues=dd.selector)

A Jupyter Widget